In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset,DataLoader
from transformers import AutoTokenizer,AutoConfig,AutoModel,Trainer
from tqdm import tqdm,trange,tqdm_notebook
from tqdm.notebook import tqdm as notetqdm
from torch.autograd import Variable
from transformers import TrainingArguments
import random
import heapq

In [3]:
MODEL_NAME = 'dccuchile/bert-base-spanish-wwm-cased'
MAX_LENGTH = 512
label_set = ['WORLD NEWS', 'ARTS & CULTURE', 'WEDDINGS', 'PARENTING', 
                 'BUSINESS & FINANCES', 'HOME & LIVING', 'EDUCATION', 
                 'WELLNESS', 'POLITICS', 'WOMEN', 'IMPACT', 'ENVIRONMENT', 
                 'SPORTS', 'FOOD & DRINK', 'GROUPS VOICES', 'MEDIA', 
                 'SCIENCE & TECH', 'CRIME', 'WEIRD NEWS', 'COMEDY', 
                 'RELIGION', 'MISCELLANEOUS', 'DIVORCE', 'ENTERTAINMENT',
                 'STYLE & BEAUTY', 'TRAVEL']

In [4]:
class BertForSequenceClassification(nn.Module):
    def __init__(self,model_name,num_classes=None):
        super().__init__()
        config = AutoConfig.from_pretrained(model_name,num_labels=num_classes)

        self.bert = AutoModel.from_pretrained(model_name,config=config)
        self.MLP =nn.Linear(config.hidden_size,num_classes)
        
    def forward(self,features,attention_mask=None,head_mask=None):
        
        assert attention_mask is not None,'attention_mask is none'
        bert_output = self.bert(input_ids=features,
                               attention_mask=attention_mask,
                               head_mask=head_mask)
    
        hidden_state = bert_output[0]

        pool_output = hidden_state[:,0]
        # print(pool_output)
        # print(pool_output.shape)
        logits = self.MLP(pool_output)
        #logits.unsqueeze(1)
        return logits

def weight_init(m):
    if isinstance(m,nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        torch.nn.init.constant_(m.bias,0)

In [5]:
model = BertForSequenceClassification(MODEL_NAME,len(label_set))
state_dict = torch.load('./results/lr3e-5/12-15-epoch-5.pth')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [7]:
content = "Talentos como Ai Haruna y Kayo Sato están resultando ser LGBT. GENKING ha anunciado que es gay. Además, entre los comediantes, Miyuki Torii ha confesado ser lesbiana, y Egashira 2:50 y Kazureza, una superaleación de arce, están resultando bisexuales. Además, en el extranjero, las actrices Jodie Foster y Elliot Page se están volviendo lesbianas. Figuras culturales como el CEO de Apple, Tim Cook, y el cantautor Elton John, también se están declarando homosexuales. Muchos atletas en el mundo del deporte están saliendo de LGBT. En los Juegos Olímpicos de Río 2016, una mujer del torneo hizo una propuesta pública a la jugadora brasileña Isadora Serlo en la sede de las chicas de rugby de siete. En los Juegos Olímpicos de Pyeongchang 2018, también se habló de que la escena en la que Gus Kenworthy, un esquiador de estilo libre estadounidense, besó al hombre de su amante, fue transmitida en vivo."

In [8]:
def predict(model,tkzr,content):
    encoded_content = tkzr(content,add_special_tokens=True,padding='max_length',
                           max_length=MAX_LENGTH,truncation=True,return_tensors='pt')
    model.eval()
    sums = 0
    with torch.no_grad():
        y_pred = model(encoded_content['input_ids'],encoded_content['attention_mask'])
        sfm = nn.Softmax(dim=1)
        ys = sfm(y_pred)[0].tolist()
        pairs = []
        for label,per in zip(label_set,ys):
            pairs.append([label,per*100])
        pairs.sort(key=lambda x:x[1],reverse=True)
        return pairs[:3]

In [9]:
res = predict(model,tokenizer,content)
res

[['GROUPS VOICES', 89.45732116699219],
 ['SPORTS', 5.698367208242416],
 ['CRIME', 1.4883363619446754]]